In [1]:
#automatically reload modules when they have changed
%reload_ext autoreload
from pathlib import Path
import random
import os

from yaml_config_wrapper import Configuration
from RLcraft import MalmoMazeEnv

In [2]:
yml_path = 'configs/mazes.yml'
# Load YML config file
c = Configuration(config_src=yml_path)
# Load configs from config class
general_config = c.get_config('general')
train_config = c.get_config('train')[0]['config']
env_config = train_config['env_config']

Configuration file loaded successfully from path: /Users/gkos/Insync/Gdrive/Projects/UTK/Minecraft-AI/configs/mazes.yml
Configuration Tag: mazes-v1


In [12]:
try:
    env.close()
except:
    pass
# Generate a seed for maze 
print("Generating new seed ...")
maze_seed = random.randint(1, 9999)
print("Loading environment ...")
# num = random.randint(0, 8)
# mission_file = run_config["env_config"]["mission_file"].replace("*", str(num))
# from glob import glob
# mission_files = glob(run_config["env_config"]["mission_file"])
# mission_file = random.choice(mission_files)
os.environ['MALMO_XSD_PATH'] = '/Users/gkos/Insync/Gdrive/Projects/UTK/Minecraft-AI/MalmoPlatform/Schemas'

env = MalmoMazeEnv(
            mazeseed=maze_seed,
            xml=env_config["xml"],
            width=900,#env_config["width"],
            height=800,#env_config["height"],
            millisec_per_tick=env_config['millisec_per_tick'],
            mission_timeout_ms=env_config['mission_timeout_ms'],
            step_reward=env_config['step_reward'],
            win_reward=env_config['win_reward'],
            lose_reward=env_config['lose_reward'],
            action_space=env_config['action_space'],
            client_port=env_config['client_port'],
            time_wait=env_config['time_wait'],
            max_loop=env_config['max_loop'])
print("Resetting environment ...")
_ = env.reset()
print("Environment Loaded!")

print("Possible actions and action space actions for current env:")
for action in range(6):
    print(f"Action: {action} -> {env.action_space[action]}")
print("Sampling an action..")
action = env.action_space.sample()
print(f'{action} -> {env.action_space[action]}')

Generating new seed ...
Loading environment ...
Resetting environment ...
Environment Loaded!
Possible actions and action space actions for current env:
Action: 0 -> move 1
Action: 1 -> move -1
Action: 2 -> strafe 1
Action: 3 -> strafe -1
Action: 4 -> turn 1
Action: 5 -> turn -1
Sampling an action..
2 -> strafe 1


In [11]:
# Take an action and get the observation, reward, done, and info
obs, reward, done, info = env.step(0)
print("Reward: ", reward)
print("Done: ", done)
print("Info: ", info)
print(f"Info.observations: len={len(info.observations)} "\
      f"type={type(info.observations[0])}")
print(f"Observation: type={type(obs)}, shape={obs.shape}")

Reward:  -1
Done:  True
Info:  WorldState (ended): 0 obs, 0 rewards, 0 frames since last state.


IndexError: Index out of range

In [40]:
# Find Usable Data
# Transform the info observation
import json
info_obs = json.loads(info.observations[-1].text)
print(info_obs.keys())
floor_data = info_obs['floor10x10']
time_data = info_obs['TotalTime']
xpos_data = info_obs['XPos']
ypos_data = info_obs['YPos']
zpos_data = info_obs['ZPos']
yaw_data = info_obs['Yaw']  # where the player is facing
hp_data = info_obs['Life']
print("# ------ Usable data ------ #")
print(f"Var obs: a {obs.shape} np array")
print(f"Var info_obs['floor10x10']: a {len(floor_data)} list: {floor_data[:3]} ... ")
print(f"Var info_obs['TotalTime']: {time_data}")
print(f"Var info_obs['XPos']: {xpos_data}")
print(f"Var info_obs['YPos']: {ypos_data}")
print(f"Var info_obs['ZPos']: {zpos_data}")
print(f"Var info_obs['Yaw']: {yaw_data}")
print(f"Var info_obs['Life']: {hp_data}")
# print(obs)

dict_keys(['DistanceTravelled', 'TimeAlive', 'MobsKilled', 'PlayersKilled', 'DamageTaken', 'DamageDealt', 'Life', 'Score', 'Food', 'XP', 'IsAlive', 'Air', 'Name', 'XPos', 'YPos', 'ZPos', 'Pitch', 'Yaw', 'WorldTime', 'TotalTime', 'floor10x10'])
# ------ Usable data ------ #
Var obs: a (84, 84, 3) np array
Var info_obs['floor10x10']: a 81 list: ['air', 'air', 'air'] ... 
Var info_obs['TotalTime']: 56363
Var info_obs['XPos']: 4.5
Var info_obs['YPos']: 227.0625
Var info_obs['ZPos']: 1.5
Var info_obs['Yaw']: 270.0
Var info_obs['XP']: 20.0


In [ ]:
# Use the internal actions to move the agent
# move: walks forwards/backwards
#   0 -> Nothing, 1 -> Forward, -1 -> Backward
# strafe: walks left/right
#   0 -> Nothing, 1 -> Right, -1 -> Left
# turn: turns the camera left/right without moving
#   0 -> Nothing, 1 -> Right, -1 -> Left
env.agent_host.sendCommand("turn 1")

In [38]:
env.close()